# Convolutional Neural Network using pytorch

This will go over my implementation of All-CNN-C model, introduced in paper [Striving For Simplicity: The All Convolutional Net](https://arxiv.org/abs/1412.6806), using pytorch library

In usual CNN, 3 types of layers are used
- Convolution Layer
- Pooling Layer
- Fully Connected Layer

This paper present All-CNN-C convolutional network, which utilizes 
- **Convolution with 2 strides** instead of MaxPool
- **Global Averaging and softmax** instead of Fully Connected Layer

### Architecture 

- First Conv Layers
    * 3 x 3 Conv, ReLU 96, Stride 1
    * 3 x 3 Conv, ReLU 96, Stride 1
- First Conv Pooling Layer
    * 3 x 3 Conv, ReLU 96, stride 2

| Layer | Kernel | Stride | Image Size | 
|-------|--------|--------|
|  Conv, ReLU 96 | 3 x 3  | 1 x 1  | 32 x 32 | 
|  Conv, ReLU 96 | 3 x 3  | 1 x 1  | 
|  **Conv, ReLU 96** | **3 x 3**  | **2 x 2**  |
|  Conv, ReLU 192 | 3 x 3  | 1 x 1  |
|  Conv, ReLU 192 | 3 x 3  | 1 x 1  |
|  **Conv, ReLU 192** | **3 x 3**  | **2 x 2**  |
|  Conv, ReLU 192 | 3 x 3  | 1 x 1  |
|  Conv, ReLU 192 | 1 x 1  | 1 x 1  | 
|  Conv, ReLU 10 | 1 x 1  | 1 x 1  | 6 x 6 |
| **Global Average** |  **6 x 6**  | **1 x 1** |
|           10 Way Softmax          |


- Batch Normalization was applied for each layers, except the first Conv layer

